In [2]:
# Importing modules
import pandas as pd
import os
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.corpora as corpora
from pprint import pprint
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
# from wordcloud import WordCloud


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/siddhartha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data = pd.read_csv('../movies_filled_cleaned_processed.csv')

data.head()

,Title,Movie Info,Distributor,Release Date,Domestic Sales (in $),International Sales (in $),World Sales (in $),Genre,Movie Runtime,License,release_date_filled,release_year,running_time_mins
0,Star Wars: Episode VII - The Force Awakens (2015),"As a new threat to the galaxy rises, Rey, a de...",Walt Disney Studios Motion Pictures,"December 16, 2015",936662225,1132859475,2069521700,"['Action', 'Adventure', 'Sci-Fi']",2 hr 18 min,PG-13,"December 16, 2015",2015,138
1,Avengers: Endgame (2019),After the devastating events of Avengers: Infi...,Walt Disney Studios Motion Pictures,"April 24, 2019",858373000,1939128328,2797501328,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13,"April 24, 2019",2019,181
2,Avatar (2009),A paraplegic Marine dispatched to the moon Pan...,Twentieth Century Fox,"December 16, 2009",760507625,2086738578,2847246203,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",2 hr 42 min,PG-13,"December 16, 2009",2009,162
3,Black Panther (2018),"T'Challa, heir to the hidden but advanced king...",Walt Disney Studios Motion Pictures,NaN,700426566,647171407,1347597973,"['Action', 'Adventure', 'Sci-Fi']",2 hr 14 min,NaN,"January 29, 2018",2018,134
4,Avengers: Infinity War (2018),The Avengers and their allies must be willing ...,Walt Disney Studios Motion Pictures,NaN,678815482,1369544272,2048359754,"['Action', 'Adventure', 'Sci-Fi']",2 hr 29 min,NaN,"April 23, 2018",2018,149


In [5]:
text_data = data[["Movie Info"]]

In [6]:
# Remove punctuation
text_data['Movie_Info_clean'] = \
text_data['Movie Info'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
text_data['Movie_Info_clean'] = \
text_data['Movie_Info_clean'].map(lambda x: x.lower())
# Print out the first rows of papers
text_data['Movie_Info_clean'].head()

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
/var/folders/24/vwrfpr75235dkb1pp5hwqn100000gn/T/ipykernel_98905/2978084983.py:3: DeprecationWarning: invalid escape sequence \.
  text_data['Movie Info'].map(lambda x: re.sub('[,\.!?]', '', x))
/var/folders/24/vwrfpr75235dkb1pp5hwqn100000gn/T/ipykernel_98905/2978084983.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_data['Movie_Info_clean'] = \
/var/folders/24/vwrfpr75235dkb1pp5hwqn100000gn/T/ipykernel_98905/2978084983.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

0    as a new threat to the galaxy rises rey a dese...
1    after the devastating events of avengers: infi...
2    a paraplegic marine dispatched to the moon pan...
3    t'challa heir to the hidden but advanced kingd...
4    the avengers and their allies must be willing ...
Name: Movie_Info_clean, dtype: object

In [7]:
#pip install pyLDAvis

In [8]:
stop_words = stopwords.words('english')
stop_words.extend(['us','get','one','two','stop','from', 'subject', 're', 'for','then','must'])
def sent_to_words(para):
    for p in para:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(p), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
info = text_data.Movie_Info_clean.values.tolist()
words = list(sent_to_words(info))
# remove stop words
words = remove_stopwords(words)
words[0][1:10]

['threat',
 'galaxy',
 'rises',
 'rey',
 'desert',
 'scavenger',
 'finn',
 'ex',
 'stormtrooper']

In [9]:
id2word = corpora.Dictionary(words)

texts = words

corpus = [id2word.doc2bow(text) for text in texts]


In [10]:


num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.006*"young" + 0.005*"new" + 0.005*"find" + 0.004*"world" + 0.004*"war" + '
  '0.004*"family" + 0.004*"life" + 0.004*"save" + 0.003*"take" + '
  '0.003*"becomes"'),
 (1,
  '0.008*"world" + 0.005*"three" + 0.004*"family" + 0.003*"boy" + 0.003*"jack" '
  '+ 0.003*"cia" + 0.003*"team" + 0.003*"son" + 0.003*"help" + 0.003*"school"'),
 (2,
  '0.005*"new" + 0.003*"three" + 0.003*"friends" + 0.003*"power" + '
  '0.003*"city" + 0.003*"men" + 0.003*"father" + 0.003*"time" + '
  '0.003*"brother" + 0.003*"lives"'),
 (3,
  '0.007*"world" + 0.006*"man" + 0.005*"group" + 0.004*"new" + 0.004*"order" + '
  '0.004*"story" + 0.004*"evil" + 0.003*"becomes" + 0.003*"war" + '
  '0.003*"save"'),
 (4,
  '0.005*"family" + 0.005*"world" + 0.005*"life" + 0.004*"woman" + 0.003*"new" '
  '+ 0.003*"city" + 0.003*"team" + 0.003*"man" + 0.003*"band" + '
  '0.003*"begins"'),
 (5,
  '0.006*"family" + 0.005*"world" + 0.005*"life" + 0.005*"young" + 0.005*"new" '
  '+ 0.004*"earth" + 0.004*"home" + 0.003*"old" +

In [11]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/Users/siddhartha/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/siddhartha/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/siddhartha/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/siddhartha/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/siddhartha/opt/

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.016413 -0.100241       1        1  12.882991
9     -0.096717 -0.044540       2        1  11.763357
3     -0.039674  0.095041       3        1  11.173702
0     -0.010855  0.062116       4        1  11.121365
5     -0.059226  0.037648       5        1  10.344158
6      0.028583 -0.007144       6        1   9.142433
8      0.083037  0.028369       7        1   9.117210
2      0.088857 -0.012134       8        1   8.645308
4     -0.006914 -0.038134       9        1   7.950363
1      0.029322 -0.020983      10        1   7.859114, topic_info=        Term       Freq      Total Category  logprob  loglift
202    group  25.000000  25.000000  Default  30.0000  30.0000
566    three  25.000000  25.000000  Default  29.0000  29.0000
25      help  36.000000  36.000000  Default  28.0000  28.0000
49     world  72.000000  72.000000  Default  27.0000  27.0000
230    woman  32.000000  32.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
41   mission   2.355659  15.306843  Topic10  -6.1670   0.6720
181    young   2.571646  49.021007  Topic10  -6.0793  -0.4042
102     love   2.396084  25.005002  Topic10  -6.1500   0.1983
317   forced   2.271933  18.076826  Topic10  -6.2032   0.4695
84       man   2.271918  50.637502  Topic10  -6.2032  -0.5606

[806 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
3471      9  0.709676    abby
95        1  0.232759  aboard
95        3  0.232759  aboard
95        6  0.232759  aboard
95        9  0.465517  aboard
...     ...       ...     ...
181       8  0.040799   young
181       9  0.040799   young
181      10  0.061198   young
2424      8  0.472714  yuppie
2349      3  0.682302    zero

[1872 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 10, 4, 1, 6, 7, 9, 3, 5, 2])

In [12]:
pyLDAvis.save_html(vis, 'lda.html')